# Data grabbing notebook
This notebook contains the code for grabbing the data for the industry portfolio returns from the Fama&French website and the currency pairs of chosen cryptocurrencies in relation to USD from Yahoo finance. Data from the latter also gets transformed into an equivalent csv file of ordinary returns. The main parameters for the two scripts are put here in the beginning for your convenience. 

In [14]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import yfinance as yf

start_date = datetime(2017, 11, 9)
end_date = datetime(2023, 9, 30)

# Creating data folders


data_industry = '12_Industry_portfolios_Daily'
#data_industry = 
#data_industry = 
#data_industry = 

tickers_crypto = ["BTC-USD", "ETH-USD", "XRP-USD"] #add more tickers here
#tickers_crypto = ["BTC-USD", "ETH-USD", "XRP-USD", "BNB-USD"] 
#tickers_crypto = ["BTC-USD", "ETH-USD", "XRP-USD", "BNB-USD", "USDT-USD"]

In [15]:
#Making folders for data raw, processed and interim data
directory_path = '../../data/raw'
directory_path = '../../data/interim'
directory_path = '../../data/processed'
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

### Industry portfolios from Ken French data library

In [16]:
# change this dummy variable to 0 for value weighted returns
equal_weight = 1 

# grabbing the data
industry_portfolios = web.DataReader(data_industry, 'famafrench', start=start_date, end = end_date)

# We grabbed a dictionary of lists, key 0,1,2 = (avg vw returns), (avg equal w returns), (DESCR) 
df = industry_portfolios[equal_weight]/100

df.to_csv(f'../../data/raw/industry_returns.csv')

# this uses date_parser instead of date_format so is suboptimal speed wise. With our data size, not really a problem.
# But this approach will be depreciated in future versions. 


c:\ProgramData\Anaconda3\lib\site-packages\pandas_datareader\famafrench.py:114: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = read_csv(StringIO("Date" + src[start:]), **params)
c:\ProgramData\Anaconda3\lib\site-packages\pandas_datareader\famafrench.py:114: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = read_csv(StringIO("Date" + src[start:]), **params)


### Crypto returns from yahoo finance

In [17]:
# empty df for storing values
crypto_returns = pd.DataFrame()

# loop for grabbing returns for provided tickers
for ticker in tickers_crypto:
    data = yf.download(ticker, start=start_date, end=end_date)
    
    # computing log returns w adjusted closing prices
    simple_return = data['Adj Close'].pct_change()
    
    # converting to df and naming column
    simple_return = simple_return.to_frame(name=ticker) 

    if crypto_returns.empty: # handles first iteration
        crypto_returns = simple_return 
    else: # i.e. the subsequent iterations
        crypto_returns = crypto_returns.join(simple_return, how="outer")

crypto_returns = crypto_returns[1:] # removing first row (NaN)

crypto_returns.to_csv('../../data/raw/crypto_returns.csv')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
